## Upload data under data/ and hts_utils.py under utils/

This notebook was heavily modified from here:

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/NonNegativeReconciliation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %%capture
# !pip install hierarchicalforecast statsforecast

In [1]:
import numpy as np
import pandas as pd

from utils.hts_eda_utils import *

from hierarchicalforecast.utils import HierarchicalPlot
from statsforecast.models import * # ARIMA, ETS, etc.
from statsforecast.core import StatsForecast

# TODO TopDown() reconciler causes KeyError 'ETS, Naive'. Same with Empirical Risk Minimization. Why?
from hierarchicalforecast.methods import * # Reconcialiation methods: BottomUp, TopDown, MinTrace etc.
from hierarchicalforecast.core import HierarchicalReconciliation

from hierarchicalforecast.evaluation import HierarchicalEvaluation

C:\Users\ltolosi-ha\AppData\Local\anaconda3\lib\site-packages\statsforecast\core.py:25: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# dataset subset to use? # Use full initially
#     deal_w_zeros_method = remove_zero_columns(df, any_or_all='any')

SELECT_TOP_K_PRODUCTS = None # None = keep all


# CHOOSE TIME SERIES METHODS HERE! https://nixtla.github.io/statsforecast/src/core/models_intro.html
TSModels = [
    ETS(season_length=7, model='ZAA'),
    Naive(),
    AutoETS(season_length=7, model='ZAA'), # I think this is newer version of ETS()
    ARIMA(),
    SeasonalExponentialSmoothingOptimized(season_length=7),
    AutoRegressive(lags=6),
    RandomWalkWithDrift()
    ]

# https://nixtla.github.io/hierarchicalforecast/methods.html
reconciliation_methods = [
    BottomUp(),
    TopDown(method='forecast_proportions'), # 'average_proportions' causes KeyError below
    MinTrace(method='wls_struct'), # Ols seems to not converge (SVD error)
    OptimalCombination(method='wls_struct'), # Same
    # ERM(method='closed') # Empirical Risk Minimization - KeyError
]

TIME_SERIES_FREQ = 'M'
df = pd.read_excel('data/Quarterly_smoothing.xlsx', index_col=0)#.iloc[:,:5])

C:\Users\ltolosi-ha\AppData\Local\anaconda3\lib\site-packages\statsforecast\models.py:841: FutureWarning: `ETS` will be deprecated in future versions of `StatsForecast`. Please use `AutoETS` instead.
  ETS._warn()


In [3]:
dataset_hierarchy_delimiter = ' - ' # The delimiter currently used in the dataset
HIERARCHY_DELIMITER = '_' # '_' is needed by HierarchicalForecast. Need to replace

## 1. Load Data

In [4]:
df.columns = df.columns.str.replace(' - ', HIERARCHY_DELIMITER) # Replace Hierarchy delimiter

##### Columns of all zeros cause errors (Division by zero in Covariance calc.). Need to fix

In [5]:
# TODO make this transform a parameter too
df = add_1_to_all_df_cells(df)

df.head()

,Дальневосточный ФО_ADRIANOL_Adrianol for adults nasal drops 10 ml #1,Дальневосточный ФО_AGALATES_Agalates tabs 0.5 mg #2,Дальневосточный ФО_AGALATES_Agalates tabs 0.5 mg #8,Дальневосточный ФО_ALMAGEL_Almagel A susp 170 ml #1,Дальневосточный ФО_ALMAGEL_Almagel Neo sachet 10 ml #10,Дальневосточный ФО_ALMAGEL_Almagel Neo susp 170 ml #1,Дальневосточный ФО_ALMAGEL_Almagel sachet 10 ml #10,Дальневосточный ФО_ALMAGEL_Almagel susp 170 ml #1,Дальневосточный ФО_ALMONT_Almont FC tabs 10 mg #28,Дальневосточный ФО_ALMONT_Almont chew tabs 4 mg #28,...,Южный ФО_VELBINE_Velbine solution for inf 10 mg/ml 5ml #1,Южный ФО_VESTIBO_Vestibo tabs 16 mg #30,Южный ФО_VESTIBO_Vestibo tabs 24 mg #30,Южный ФО_VINCRISTINE-TEVA_Vincristine-Teva lyoph for inf 1 mg/ml 1 ml #1,Южный ФО_VINCRISTINE-TEVA_Vincristine-Teva lyoph for inf 1 mg/ml 2 ml #1,Южный ФО_VINORELBINE-TEVA_VINORELBIN-TEVA 50 mg.5 ml,Южный ФО_VINORELBINE-TEVA_VINORELBINE-TEVA concentrate 10 mg.ml 1 ml,Южный ФО_ZINCTERAL_Zincteral-Teva FC tabs 124 mg #150,Южный ФО_ZINCTERAL_Zincteral-Teva FC tabs 124 mg #25,Южный ФО_ZOLEDRONAT-TEVA_Zoledronate-Teva concentrate for inf 4 mg/5ml 5 ml #1
Month,,,,,,,,,,,,,,,,,,,,,
2018-03-01,200,1,1,10,1,1,1,949,36,88,...,106,1,315,401,1,1,1,1,17,9690
2018-04-01,1000,1,1,10,1,1,1,1037,36,88,...,1046,1,248,2161,1,1,1,1,1,11705
2018-05-01,1731,1,1,302,1,1,1,1246,94,148,...,1011,1,248,3246,1,1,1,1,1,15233
2018-06-01,2090,1,1,491,1,1,1,1787,184,104,...,947,1,1,3236,1,1,1,1,1,6586
2018-07-01,1547,1,61,491,1,1,1,4132,184,104,...,46,1,1,5086,1,1,1,7,1,6035


##### Optional: Select only top Products

Saves compute

In [7]:
# if SELECT_TOP_K_PRODUCTS is not None:
#     df = select_top_n_brands(df, n=SELECT_TOP_K_PRODUCTS)

# # df.head(5)
brand_name = 'ALMAGEL'
df_brand = select_brand(df, brand_name, HIERARCHY_DELIMITER='_')
df_brand.columns = [c.replace(brand_name + HIERARCHY_DELIMITER, "") for c in df_brand.columns]

In [8]:
df

,Дальневосточный ФО_Almagel Neo sachet 10 ml #10,Дальневосточный ФО_Almagel Neo susp 170 ml #1,Дальневосточный ФО_Almagel sachet 10 ml #10,Дальневосточный ФО_Almagel susp 170 ml #1,Приволжский ФО_Almagel A sachet 10 ml #10,Приволжский ФО_Almagel A susp 170 ml #1,Приволжский ФО_Almagel Neo sachet 10 ml #10,Приволжский ФО_Almagel Neo susp 170 ml #1,Приволжский ФО_Almagel sachet 10 ml #10,Приволжский ФО_Almagel susp 170 ml #1,...,Центральный ФО_Almagel Neo sachet 10 ml #10,Центральный ФО_Almagel Neo susp 170 ml #1,Центральный ФО_Almagel sachet 10 ml #10,Центральный ФО_Almagel susp 170 ml #1,Южный ФО_Almagel A sachet 10 ml #10,Южный ФО_Almagel A susp 170 ml #1,Южный ФО_Almagel Neo sachet 10 ml #10,Южный ФО_Almagel Neo susp 170 ml #1,Южный ФО_Almagel sachet 10 ml #10,Южный ФО_Almagel susp 170 ml #1
Month,,,,,,,,,,,,,,,,,,,,,
2018-03-01,1,1,1,949,1,191,1,5,1,1211,...,1,5,1,3133,1,70,1,1,1,1174
2018-04-01,1,1,1,1037,1,266,1,25,1,2233,...,1,137,1,17738,1,167,1,1,1,1841
2018-05-01,1,1,1,1246,1,279,1,23,1,2503,...,14,136,1,29416,1,112,8,51,1,2055
2018-06-01,1,1,1,1787,1,322,11,27,1,3146,...,14,141,1,45994,1,5116,8,51,1,3365
2018-07-01,1,1,1,4132,1,330,11,172,1,3251,...,14,609,1,44251,1,5069,38,71,1,4125
2018-08-01,1,1,1,4509,1,154,11,175,1,3594,...,1,606,1,38742,1,10205,31,21,1,4109
2018-09-01,1,1,1,3761,1,303,31,173,1,2911,...,1,611,1,24302,1,5200,31,21,1,3049
2018-10-01,1,1,1,2281,1,697,31,8,1,2544,...,5,3360,1,25942,1,5190,1,1,1,1917
2018-11-01,1,1,1,2095,1,792,31,27,1,3197,...,8,3375,1,30274,1,86,1,1,1,1699


In [9]:
%%capture
df_with_aggregates, hierarchy = prep_data_for_scikit_hts(df)

<font color='cyan'>HierarchicalForecast likes data to be Drug | Date | Sales, rather than having DrugName as columns</font>


### Melt data into format required by HierarchicalForecast

Following how their example code's data looks

In [10]:
# Melt the DataFrame - convert ColNames to rows to match input to HierForecast
df_with_aggregates.reset_index(inplace=True) # Move Month index to column (package requirement)

# TODO Check these for prediction error
melted_df = df_with_aggregates.melt(id_vars=['Month'], var_name='Drug', value_name='Sales')

# Convert melted DataFrame to the required format
melted_df = melted_df[['Drug', 'Month', 'Sales']]

# Col names seem to need to be thus for package
melted_df.rename(columns={'Drug': 'unique_id', 'Month':'ds', 'Sales':'y'}, inplace=True)


print(melted_df.head())
print(melted_df.tail())


                                         unique_id         ds    y
0  Дальневосточный ФО_Almagel Neo sachet 10 ml #10 2018-03-01  1.0
1  Дальневосточный ФО_Almagel Neo sachet 10 ml #10 2018-04-01  1.0
2  Дальневосточный ФО_Almagel Neo sachet 10 ml #10 2018-05-01  1.0
3  Дальневосточный ФО_Almagel Neo sachet 10 ml #10 2018-06-01  1.0
4  Дальневосточный ФО_Almagel Neo sachet 10 ml #10 2018-07-01  1.0
     unique_id         ds        y
5638     Total 2022-07-01  84758.0
5639     Total 2022-08-01  29540.0
5640     Total 2022-09-01  64278.0
5641     Total 2022-10-01  67224.0
5642     Total 2022-11-01  82248.0


### Creating `S_df`

All colored font is Ariel

<font color='turquoise'>We've created `Y_df, tags`. All we need is `S_df`</font>
This is like a tree representing the hierarchy, with aggregations at each level

<font color='blue'>`S_df` is a representation of the Hierarchy - 1 means that column name (item, Drugs in our case), belongs to the Total row. Rows represent totals at each level of the hierarchy, for each node</font>

In [11]:
S_df = create_S_df(df)

S_df.head()

,Дальневосточный ФО_Almagel Neo sachet 10 ml #10,Дальневосточный ФО_Almagel Neo susp 170 ml #1,Дальневосточный ФО_Almagel sachet 10 ml #10,Дальневосточный ФО_Almagel susp 170 ml #1,Приволжский ФО_Almagel A sachet 10 ml #10,Приволжский ФО_Almagel A susp 170 ml #1,Приволжский ФО_Almagel Neo sachet 10 ml #10,Приволжский ФО_Almagel Neo susp 170 ml #1,Приволжский ФО_Almagel sachet 10 ml #10,Приволжский ФО_Almagel susp 170 ml #1,...,Центральный ФО_Almagel Neo sachet 10 ml #10,Центральный ФО_Almagel Neo susp 170 ml #1,Центральный ФО_Almagel sachet 10 ml #10,Центральный ФО_Almagel susp 170 ml #1,Южный ФО_Almagel A sachet 10 ml #10,Южный ФО_Almagel A susp 170 ml #1,Южный ФО_Almagel Neo sachet 10 ml #10,Южный ФО_Almagel Neo susp 170 ml #1,Южный ФО_Almagel sachet 10 ml #10,Южный ФО_Almagel susp 170 ml #1
Total,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Дальневосточный ФО,1,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Дальневосточный ФО_Almagel Neo sachet 10 ml #10,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Дальневосточный ФО_Almagel Neo susp 170 ml #1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Дальневосточный ФО_Almagel sachet 10 ml #10,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Create `tags`, which is a description of the Hierarchy as `dict`

Original `tags` loaded from example Dataset - they didn't create it programmatically

In [12]:
hierarchy

{'Total': ['Сибирский ФО',
  'Приволжский ФО',
  'Северо-кавказский ФО',
  'Уральский ФО',
  'Центральный ФО',
  'Дальневосточный ФО',
  'Южный ФО',
  'Северо-западный ФО'],
 'Сибирский ФО': ['Сибирский ФО_Almagel A susp 170 ml #1',
  'Сибирский ФО_Almagel A sachet 10 ml #10',
  'Сибирский ФО_Almagel sachet 10 ml #10',
  'Сибирский ФО_Almagel susp 170 ml #1',
  'Сибирский ФО_Almagel Neo susp 170 ml #1',
  'Сибирский ФО_Almagel Neo sachet 10 ml #10'],
 'Сибирский ФО_Almagel A susp 170 ml #1': [],
 'Сибирский ФО_Almagel A sachet 10 ml #10': [],
 'Сибирский ФО_Almagel sachet 10 ml #10': [],
 'Сибирский ФО_Almagel susp 170 ml #1': [],
 'Сибирский ФО_Almagel Neo susp 170 ml #1': [],
 'Сибирский ФО_Almagel Neo sachet 10 ml #10': [],
 'Приволжский ФО': ['Приволжский ФО_Almagel A susp 170 ml #1',
  'Приволжский ФО_Almagel A sachet 10 ml #10',
  'Приволжский ФО_Almagel sachet 10 ml #10',
  'Приволжский ФО_Almagel susp 170 ml #1',
  'Приволжский ФО_Almagel Neo susp 170 ml #1',
  'Приволжский ФО_

In [ ]:
# TODO wtf did chatgpt do here?
transformed_data = { # Need names for hierarchy levels IMO
    "Sales": ["Total"],
    "Sales/Region": hierarchy['Total'],
    "Sales/Region/DrugName": sum([hierarchy[region] for region in hierarchy['Total']], []),
    "Sales/Region/DrugName/DrugDosage": sum([hierarchy[key] for key in sum([hierarchy[region] for region in hierarchy['Total']], [])], []),
}

# Convert the lists to numpy arrays for consistency with the format
for key in transformed_data:
    transformed_data[key] = np.array(transformed_data[key], dtype=object)

# print(transformed_data)
tags = transformed_data

We split the dataframe in train/test splits.

In [ ]:
Y_df = melted_df

# Y_df

In [ ]:
Y_test_df = Y_df.groupby('unique_id').tail(7) # Original code
Y_train_df = Y_df.drop(Y_test_df.index)

Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

In [ ]:
print(Y_test_df.head())
print(Y_test_df.tail())

## 2. Base Forecasts

The following cell computes the *base forecast* for each time series using the `ETS` and `naive` models. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
%%capture
fcst = StatsForecast(
    df=Y_train_df,
    models=TSModels,
    # models=[ETS(season_length=7, model='ZZA'), Naive()],
    freq=TIME_SERIES_FREQ,
    n_jobs=-1
)
Y_hat_df = fcst.forecast(h=7) # TODO What is h=7?

Observe that the ETS model computes negative forecasts for some series.

<font color='pink'>Does `Y_hat_df` have a `ds` column in the original code?</font>

Yes

In [ ]:
# Make sure S_df does not have extra entries - TODO I still don't know exactly how S_df is created
#   This is jerry-rigged to work
rows_to_drop = list(set(S_df.index) - set(Y_test_df.index))
# rows_to_drop
S_df.drop(rows_to_drop, inplace=True)

In [ ]:
# `S_df` should have 1 entry for each unique row in `Y_hat_df`
assert(len(S_df.index) == len(set(Y_hat_df.index)))
assert(set(Y_train_df.index) - set(S_df.index) == set())
assert(set(S_df.index) - set(Y_train_df.index) == set())

## 3. Non-Negative Reconciliation

The following cell makes the previous forecasts coherent and nonnegative using the `HierarchicalReconciliation` class.

In [ ]:
hrec = HierarchicalReconciliation(reconcilers=reconciliation_methods)


Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df,
                          S=S_df, tags=tags)

Y_rec_df.head()

Observe that the nonnegative reconciliation method obtains nonnegative forecasts.

## 4. Evaluation

The `HierarchicalForecast` package includes the `HierarchicalEvaluation` class to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
# TODO enhance this
def mse(y, y_hat):
    return np.mean((y-y_hat)**2)

evaluator = HierarchicalEvaluation(evaluators=[mse, mean_absolute_percentage_error, symmetric_mean_absolute_percentage_error])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df,
        tags=tags#, benchmark='Naive'
)
evaluation#.filter(like='ETS', axis=1).T

Observe that the nonnegative reconciliation method performs better that its unconstrained counterpart.

## Plot Hierarchy & Evaluations

In [ ]:
a = Y_test_df.sort_index()#.sort_values(by='ds', ascending=True)

# TODO programmatically get these by subtracting column names (set)
b = Y_rec_df[['ETS', 'Naive', 'ETS/BottomUp', 'Naive/BottomUp']].sort_index()#.sort_values(by='ds', ascending=True)#.drop(columns=['ds'])

b

In [ ]:
merged_test_preds_df = pd.concat([a, b], axis=1)
# merged_test_preds_df

In [ ]:
merged_test_preds_df = merged_test_preds_df.sort_values(by='ds', ascending=True)
merged_test_preds_df

In [ ]:
hplt = HierarchicalPlot(S=S_df, tags=tags)

hplt.plot_hierarchical_predictions_gap(Y_df=merged_test_preds_df, models = 'ETS')#['ETS', 'Naive', 'ETS/BottomUp', 'Naive/BottomUp'])

In [ ]:
hplt.plot_hierarchically_linked_series(bottom_series='Южный ФО_VALZ_Valz N FC tabs 80 mg/12.5mg #28', Y_df=Y_train_df)